<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [3]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [5]:
#ANSWER
response.status_code

200

In [7]:
#ANSWER
print(response.text)

{"iss_position": {"longitude": "159.3650", "latitude": "14.8492"}, "timestamp": 1752850345, "message": "success"}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [9]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [11]:
#ANSWER
print(response.headers)

{'Date': 'Fri, 18 Jul 2025 14:52:43 GMT', 'Server': 'Apache', 'X-Amzn-Trace-Id': 'Root=1-687a5fbb-8893eded454817adfce11201;', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [13]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2025-07-18T14:52:43.421952614Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2025-07-17T21:20:49.892964932Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":13,"passes":[{"start":"2025-07-18T16:33:43.409Z","tca":"2025-07-18T16:37:13.409Z","end":"2025-07-18T16:40:33.409Z","aos_azimuth":18,"los_azimuth":96,"max_elevation":5.0},{"start":"2025-07-18T18:07:48.409Z","tca":"2025-07-18T18:13:18.409Z","end":"2025-07-18T18:18:43.409Z","aos_azimuth":316,"los_azimuth":131,"max_elevation":78.0},{"start":"2025-07-18T19:45:38.409Z","tca":"2025-07-18T19:50:08.409Z","end":"2025-07-18T19:55:03.409Z","aos_azimuth":267,"los_azimuth":148,"max_elevation":13.0},{"start":"2025-07-18T21:25:23.409Z","tca":"2025-07-18T21:28:23.409Z","end":"2025-07-18T21:31:28.409Z","aos_azimuth":221,"los_azimuth":153,"max_elevation":3.0},{"start":"2025-07-18T23:03:33.409Z","tca":"2025-07-18T23:07:03.409Z","end":"2025-07-18T23:10:18.409Z","aos_azimuth

Note that this is a Python byte string:

In [15]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [17]:
#ANSWER
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [19]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2025-07-18T14:52:43.421952614Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2025-07-17T21:20:49.892964932Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 13, 'passes': [{'start': '2025-07-18T16:33:43.409Z', 'tca': '2025-07-18T16:37:13.409Z', 'end': '2025-07-18T16:40:33.409Z', 'aos_azimuth': 18, 'los_azimuth': 96, 'max_elevation': 5.0}, {'start': '2025-07-18T18:07:48.409Z', 'tca': '2025-07-18T18:13:18.409Z', 'end': '2025-07-18T18:18:43.409Z', 'aos_azimuth': 316, 'los_azimuth': 131, 'max_elevation': 78.0}, {'start': '2025-07-18T19:45:38.409Z', 'tca': '2025-07-18T19:50:08.409Z', 'end': '2025-07-18T19:55:03.409Z', 'aos_azimuth': 267, 'los_azimuth': 148, 'max_elevation': 13.0}, {'start': '2025-07-18T21:25:23.409Z', 'tca': '2025-07-18T21:28:23.409Z', 'end': '2025-07-18T21:31:28.409Z', 'aos_azimuth': 221, 'los_azimuth': 153, 'max_elevation': 3.0}, {'start': '2025-07-18T23:03:33.409Z', 'tca': '2

What kind of object did this give us?

In [21]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [23]:
#ANSWER:
passes = overheads['passes']

Now extract the `start` strings into an array called `srisetimes`:

In [25]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [27]:
srisetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [29]:
#ANSWER:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2025, 7, 18, 16, 33, 43, 409000),
 datetime.datetime(2025, 7, 18, 18, 7, 48, 409000),
 datetime.datetime(2025, 7, 18, 19, 45, 38, 409000),
 datetime.datetime(2025, 7, 18, 21, 25, 23, 409000),
 datetime.datetime(2025, 7, 18, 23, 3, 33, 409000),
 datetime.datetime(2025, 7, 19, 0, 39, 43, 409000),
 datetime.datetime(2025, 7, 19, 2, 16, 13, 409000),
 datetime.datetime(2025, 7, 19, 3, 56, 13, 409000),
 datetime.datetime(2025, 7, 19, 15, 47, 58, 409000)]

Finally, here is an endpoint that tells us who is on board:

In [31]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [33]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
for astronaut in astros['people']:
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [35]:
#ANSWER:

def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [37]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

Response = 200: OK


False

>

>

>



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



